In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
filefolder = r'C:\Users\johan\OneDrive\Marcobre\1.0 PROJECTS\01. Crear budget de mantenimiento\hr2csv\TECSUP\HR'
filename = r'allHRdata.csv'
mergeHRname = r'mergeHR.csv'
EQ2GHRname = r'Hoja Resumen- CARGA A SAP V09.11.xlsx'

In [3]:
HRrute = os.path.join(filefolder,filename)
mergeHRrute = os.path.join(filefolder,mergeHRname)
EQ2GHRrute = os.path.join(filefolder,EQ2GHRname)

In [4]:
HRrute

'C:\\Users\\johan\\OneDrive\\Marcobre\\1.0 PROJECTS\\01. Crear budget de mantenimiento\\hr2csv\\TECSUP\\HR\\allHRdata.csv'

In [5]:
#db1 = pd.read_excel(HRrute,sheet_name=flag, skiprows = 6, header = None)
db = pd.read_csv(HRrute)

eq2ghr = pd.read_excel(EQ2GHRrute,sheet_name=0, header=0)

In [6]:
eq2ghr.drop(columns=['OBSERVACIONES','PLAN DE MANTENIMIENTO','HOJA DE RUTA TIPO EQUIPO','HOJA DE RUTA INSTRUCCIÓN'],inplace=True)

In [7]:
eq2ghr.columns = ['TAG','DescripcionEquipo','GrupoTS','GrupoHR','EqSAP','PuntoMedida']
#eq2ghr = eq2ghr.loc[(eq2ghr.loc[:,'GrupoTS']== grupo1) | (eq2ghr.loc[:,'GrupoTS']== grupo2),:]
#eq2ghr.loc[(eq2ghr.loc[:,'EqSAP']== 'UBT','EqSAP')] = 0
#eq2ghr.loc[:,'EqSAP'] = eq2ghr.loc[:,'EqSAP'].astype(float)

In [8]:
db.columns = db.columns.astype(int)

In [9]:
# Hojas de ruta
HR = db.loc[db.iloc[:,0] == 1,:]
HR.drop(labels=[11,12,13,14,15,16,17], axis = 1, inplace = True)
HRcolumnsname = [0,'GrupoHR','Contador','HRDescripcion','HRPuesto','Centro','Utilizacion','GPlanificacion', 'EstadoGlobal','EstadoInstalacion','Estrategia']
HR.columns = HRcolumnsname
HR.loc[:,'HRContador'] = HR['GrupoHR'].astype(str) + HR['Contador']
# Add maintenance estrategy of each actiivitie, and type of equipment
HR.loc[:,'EstrategiaMTTO'] = HR.loc[:,'HRDescripcion'].str.split('_',expand = True)[0] 
HR.loc[:,'ClaseEq'] = HR.loc[:,'HRDescripcion'].str.split('_',expand = True)[2]

In [10]:
# Posiciones de las Hojas de Ruta
PHR = db.loc[db.iloc[:,0] == 2,:]
PHR.drop(columns = [10,11,12,13,14,15,16,17],inplace = True)
PHRcolumnsname = [0,'GrupoHR','pos','Puesto','ClaseOT','Actividades','HH','H','ACant','AHoras']
PHR.columns = PHRcolumnsname

In [11]:
# Actividad detallada de laPosiciones de las Hojas de Ruta
AHR = db.loc[db.iloc[:,0] == 3,:]

In [12]:
%%time
# Conectar Posiciones de HR con la HR a traves de HRContador del HR al PHR
nm1=0
#HR.tail(1).index.values[0] = last index of HR dataframe
for m in PHR.index:
    for n in HR.index:
        if nm1<=m and m<=n:
            PHR.loc[m,'HRContador'] = HR.loc[nm1,'HRContador']
            break
        elif m>=HR.tail(1).index.values[0]:
            PHR.loc[m,'HRContador'] = HR.loc[HR.tail(1).index.values[0],'HRContador']
            break
        nm1 = n
PHR.loc[:,'posHRContador'] = PHR.loc[:,'HRContador'] + PHR.loc[:,'pos'].astype(str)

Wall time: 30min 19s


In [13]:
# Materiales de Hojas de Ruta
MHR = db.loc[db.iloc[:,0] == 4,:]
MHR = MHR.drop(columns = [4,5,6,7,8,9,10,11,12,13,14,15,16,17])
MHRcolumnsname = [0,'GrupoHR','Material','MCantidad']
MHR.columns = MHRcolumnsname

In [14]:
%%time
# Conectar Posiciones de HR con la HR a traves de posHRContador del PHR al MHR
nm1=0
#HR.tail(1).index.values[0] = last index of HR dataframe
for m in MHR.index:
    for n in PHR.index:
        if nm1<=m and m<=n:
            MHR.loc[m,'posHRContador'] = PHR.loc[nm1,'posHRContador']
            MHR.loc[m,'HRContador'] = PHR.loc[nm1,'HRContador']
            break
        elif m>=PHR.tail(1).index.values[0]:
            MHR.loc[m,'posHRContador'] = PHR.loc[PHR.tail(1).index.values[0],'posHRContador']
            MHR.loc[m,'HRContador'] = PHR.loc[PHR.tail(1).index.values[0],'HRContador']
            break
        nm1 = n
#PHR.loc[:,'posHRContador'] = PHR.loc[:,'HRContador'] + PHR.loc[:,'pos'].astype(str)

Wall time: 8min 30s


In [15]:
# Materiales de Estrategia de Ruta
EPHR = db.loc[db.iloc[:,0] == 5,:]
EPHR.drop(columns = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],inplace = True)
EPHR.columns = [0,'GrupoHR','Frecuencia']

In [19]:
%%time
# Conectar Estrategias de HR con la posicion de la HR
nm1=0
#HR.tail(1).index.values[0] = last index of HR dataframe
for m in EPHR.index:
    for n in PHR.index:
        if nm1<=m and m<=n:
            EPHR.loc[m,'posHRContador'] = PHR.loc[nm1,'posHRContador']
            #EPHR.loc[m,'HRContador'] = PHR.loc[nm1,'HRContador']
            break
        elif m>=PHR.tail(1).index.values[0]:
            EPHR.loc[m,'posHRContador'] = PHR.loc[PHR.tail(1).index.values[0],'posHRContador']
            #EPHR.loc[m,'HRContador'] = PHR.loc[HR.tail(1).index.values[0],'HRContador']
            break
        nm1 = n
#PHR.loc[:,'posHRContador'] = PHR.loc[:,'HRContador'] + PHR.loc[:,'pos'].astype(str)

Wall time: 43min 38s


In [20]:
PHRE = pd.merge(PHR.loc[:,['pos','Actividades','Puesto','ClaseOT','ACant','AHoras','HRContador','posHRContador']],EPHR.loc[:,['Frecuencia','posHRContador']],on = 'posHRContador',how = 'left')

In [21]:
allDuties = pd.merge(HR.loc[:,['GrupoHR','Contador','HRDescripcion','HRPuesto','GPlanificacion','EstadoGlobal','EstadoInstalacion','Estrategia','HRContador','EstrategiaMTTO']], PHRE.loc[:,['pos', 'Actividades', 'Puesto', 'ClaseOT', 'ACant', 'AHoras','HRContador', 'posHRContador', 'Frecuencia']], on = 'HRContador', how = 'left')

In [22]:
allDutiesMat = pd.merge(allDuties,MHR.loc[:,['posHRContador','Material','MCantidad']],on='posHRContador',how='left')

In [23]:
HR.to_csv(filefolder + r'\HR.csv', encoding= 'utf-8')
PHRE.to_csv(filefolder + r'\PHRE.csv', encoding= 'utf-8')
allDuties.to_csv(filefolder + r'\allDuties.csv', encoding= 'utf-8')
allDutiesMat.to_csv(filefolder + r'\allDutiesMat.csv', encoding= 'utf-8')